<img src="https://whatcar.vn/media/2018/09/car-lot-940x470.jpg"/>

## Прогнозирование стоимости автомобиля по характеристикам
*Этот Ноутбук является Примером/Шаблоном (Baseline) к этому соревнованию и не служит готовым решением!*   
Вы можете использовать его как основу для построения своего решения.


> **baseline** создается больше как шаблон, где можно посмотреть как происходит обращение с входящими данными и что нужно получить на выходе. При этом МЛ начинка может быть достаточно простой. Это помогает быстрее приступить к самому МЛ, а не тратить ценное время на чисто инженерные задачи. 
Также baseline является хорошей опорной точкой по метрике. Если твое решение хуже baseline - ты явно делаешь что-то не то и стоит попробовать другой путь) 

Помним, что по условию соревнования, нам нужно самостоятельно собрать обучающий датасет. В этом ноутбуке мы не будем рассматривать сбор данных. Предположим, что мы уже все собрали и просто подключили свой датасет через "Add Data", чтобы приступить к самому ML.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

from sklearn.model_selection import train_test_split, cross_val_score, KFold 
from sklearn.linear_model import LogisticRegression


from sklearn.metrics import confusion_matrix
from sklearn.metrics import auc, roc_auc_score, roc_curve
from sklearn.ensemble import GradientBoostingRegressor, ExtraTreesRegressor, RandomForestRegressor


from scipy.stats import boxcox
from scipy.stats import yeojohnson, uniform
from sklearn.model_selection import GridSearchCV
from scipy.stats import uniform, truncnorm, randint
from pprint import pprint
import random
from catboost import CatBoostRegressor
from itertools import groupby

import sys
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor
from sklearn.ensemble import BaggingRegressor

from sklearn.metrics import recall_score
from sklearn.metrics import auc, roc_auc_score, roc_curve
from sklearn.base import clone
import datetime
from datetime import datetime, timedelta
import math

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor
from pprint import pprint

In [ ]:
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [ ]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

# Setup

In [ ]:
VERSION    = 15
DIR_TRAIN  = '../input/auto-ru-v3-30-11-2020/' # подключил к ноутбуку внешний датасет
DIR_TEST   = '../input/sf-dst-car-price-prediction/'
VAL_SIZE   = 0.20   # 20%

# CATBOOST
ITERATIONS = 5000
LR         = 0.1

# Data

In [ ]:
!ls '../input'

In [ ]:
train = pd.read_csv(DIR_TRAIN+'auto_ru_v3_30_11_2020.csv') # датасет для обучения модели
test = pd.read_csv(DIR_TEST+'test.csv')
sample_submission = pd.read_csv(DIR_TEST+'sample_submission.csv')

In [ ]:
train.sample(2)

In [ ]:
train.info()

In [ ]:
(train.shape)

In [ ]:
test.sample(2)

In [ ]:
(test.shape)

In [ ]:
#  у участников найдена функция по сравнению дата сетов. Решила ее использовать
def check_df_before_merg(d_df1,d_df2,n_line):
    
    list_of_names1 = list(d_df1.columns)
    temp_dict = {}
    temp_dict['# уник_1'] = d_df1.nunique().values
    temp_dict[f'в  строке_1 ({n_line} позиц-я)'] =d_df1.loc[n_line].values
    temp_dict['тип_1'] = d_df1.dtypes
    temp_dict['имя признака_1'] = list_of_names1
    temp_df1 = pd.DataFrame.from_dict(temp_dict)
    
    
    list_of_names2 = list(d_df2.columns)
    temp_dict2 = {}
    temp_dict2['имя признака_2'] = list_of_names2
    temp_dict2['тип_2'] = d_df2.dtypes
    temp_dict2[f'в  строке_2 ({n_line} позиц-я)'] =d_df2.loc[n_line].values
    temp_dict2['# уник_2'] = d_df2.nunique().values
    temp_df2 = pd.DataFrame.from_dict(temp_dict2)
    
    temp_df = pd.concat([temp_df1,temp_df2], axis=1, sort=False)
    temp_df.reset_index(inplace = True)
    del temp_df['index']
    display(temp_df)

    temp_dict3 = {}
    temp_df3= pd.DataFrame(temp_df)
    temp_list  = []
    temp_list2  = []
    temp_list3  = []
    temp_list4  = []
    temp_list5  = []

    for i in range(len(temp_df)):
        if str(temp_df3['тип_2'][i]) != str(temp_df3['тип_1'][i]):
            temp_list.append(temp_df3['имя признака_1'][i])
            temp_list2.append(temp_df3['имя признака_2'][i])
            temp_list3.append(str(temp_df3['тип_1'][i]) + '!=' + str(temp_df3['тип_2'][i]))
            temp_list4.append(i)
        if temp_df3['# уник_2'][i]>0 and temp_df3['# уник_1'][i]/temp_df3['# уник_2'][i] > 2:
            temp_list5.append(i)
            
    temp_dict3['index']= temp_list4
    temp_dict3['имя признака_1']= temp_list
    temp_dict3['не совпадают типы'] = temp_list3
    temp_dict3['имя признака_2']= temp_list2

    temp_df4 = pd.DataFrame.from_dict(temp_dict3)
    temp_df4.set_index('index',inplace=True)

    print(f'Резюме:\n 1. Не совпали типы в:= {len(temp_df4)} столбцах\n')
    print(f'2. Уникальные значения различаются в:= {len(temp_list5)} столбцах {temp_list5}')
    display(temp_df4)



    return

In [ ]:
len(train.index)

In [ ]:
len(test.index)

In [ ]:
check_df_before_merg(train, test,56)

Некоторые данные не удалось найти. Для работы необходимо будет удалить.
Планируется удаить колонки  car_url, sell_id, image, Состояние.

In [ ]:
train.vehicleTransmission.unique()

In [ ]:
train.fuelType.unique()

In [ ]:
train['Привод'].unique()

In [ ]:
train.priceCurrency.unique()

In [ ]:
test.vehicleTransmission.unique()

In [ ]:
test.fuelType.unique()

In [ ]:
test['Привод'].unique()

In [ ]:
# Приведем данные к одному типу
test['modelDate'] = test['modelDate'].astype('float64')
test['numberOfDoors'] = test['numberOfDoors'].astype('float64')
test['engineDisplacement'] = test['engineDisplacement'].apply(lambda x: str(x).replace('LTR', ''))

train['Состояние'] = train['Состояние'].astype('object')

train['engineDisplacement'] = train['engineDisplacement']/1000
train['enginePower'] = train['enginePower'].astype('object')

train['Таможня'] = train['Таможня'].map({True: 'Растаможен', False:'Не растаможен'})

train['bodyType'] = train['bodyType'].apply(lambda x: str(x).lower())

dict_color = {'040001':'чёрный', 'EE1D19':'красный', '0000CC':'синий', 
              'CACECB':'серебристый', '007F00':'зелёный', 'FAFBFB':'белый', 
              '97948F':'серый', '22A0F8':'голубой', '660099':'пурпурный', 
              '200204':'коричневый', 'C49648':'бежевый', 'DEA522':'золотистый', 
              '4A2197':'фиолетовый', 'FFD600':'жёлтый', 'FF8649':'оранжевый', 
              'FFC0CB':'розовый'}
train['color'] = train['color'].map(dict_color)

train['Руль'] = train['Руль'].map({'RIGHT':'Правый', 'LEFT':'Левый'})
train['vehicleTransmission'] = train['vehicleTransmission'].map({'AUTOMATIC':'автоматическая', 'MECHANICAL':'механическая',
                                                                 'ROBOT':'роботизированная', 'VARIATOR':'вариатор'})

train['fuelType'] = train['fuelType'].map({'GASOLINE':'бензин', 'DIESEL':'дизель',
                                                    'HYBRID':'гибрид', 'ELECTRO':'электро'})
train['Привод'] = train['Привод'].map({'FORWARD_CONTROL':'передний', 'ALL_WHEEL_DRIVE':'полный',
                                                    'REAR_DRIVE':'задний'})

# ПТС - тут все просто - Оригинал или Дубликат в нижнем регистре с большой буквы
train['ПТС'] = train['ПТС'].map({'ORIGINAL':'Оригинал', 'DUPLICATE':'Дубликат'})

train['priceCurrency'] = train['priceCurrency'].map({'RUR':'RUB'})

train['Владельцы'] = train['Владельцы'].map({3.0:'3 или более', 2.0:'2\xa0владельца', 1.0:'1\xa0владелец'})


In [ ]:
test['engineDisplacement']

In [ ]:
test['engineDisplacement'] = pd.to_numeric(test['engineDisplacement'], errors='coerce').astype('float64')

In [ ]:
df_train_samp = train

In [ ]:
df_train_samp['Владение']


In [ ]:
df_train_samp['Владение'] = df_train_samp['Владение'].str.replace('{','')
df_train_samp['Владение'] = df_train_samp['Владение'].str.replace('}','')
df_train_samp['Владение'] = df_train_samp['Владение'].str.replace("'",'')
df_train_samp['Владение'] = df_train_samp['Владение'].str.replace(" ",'')
df_train_samp['Владение'] = df_train_samp['Владение'].str.replace(":",',')
df_train_samp['Владение'] = df_train_samp['Владение'].str.replace(",",'|')
df_train_samp['Владение'] = df_train_samp['Владение'].str.replace("year",'y')
df_train_samp['Владение'] = df_train_samp['Владение'].str.replace("month",'m')
df_train_samp['Владение']

In [ ]:
df_train_samp['Владение']

In [ ]:
df_train_samp['Владение'] = df_train_samp['Владение'].fillna("0|0000|0|00")

In [ ]:
df2 = pd.DataFrame(df_train_samp['Владение'])

In [ ]:
df3 = pd.DataFrame(df2['Владение'].str.split('|').values.tolist())

In [ ]:
def year_current (x):
    y = -2020
    return y
df3['year_current'] = df3.apply(year_current, axis =1)

In [ ]:
df3.columns

In [ ]:
df3.rename(columns={0: 'y', 1: 'y_num', 2: 'm',3: 'm_num'}, inplace=True)

In [ ]:
df3.columns

In [ ]:
df3.drop(['y', 'm'], axis=1, inplace=True )

In [ ]:
df3['y_num'] = df3['y_num'].astype('int64')
df3['m_num'] = df3['m_num'].astype('int64')

In [ ]:
df3['n_years'] = df3.loc[df3['y_num'] > 0,['y_num','year_current']].sum(axis=1)

In [ ]:
df3['month_ownership'] = df3['n_years']*(-12)+df3['m_num']

In [ ]:
df3['month_ownership'] = df3['month_ownership'].fillna(df3['month_ownership'].median())

In [ ]:
df3

In [ ]:
train = pd.concat((train, df3['month_ownership']), axis = 1)


In [ ]:
train.sample(4)

In [ ]:
df2 = pd.DataFrame(test)

In [ ]:
    def num_of_months(row):
        if pd.notnull(row):
            list_ownership = row.split()
            if len(list_ownership) == 2:
                if list_ownership[1] in ['год', 'года', 'лет']:
                    return int(list_ownership[0])*12
                return int(list_ownership[0])
            return int(list_ownership[0])*12 + int(list_ownership[3])
    
    df2['month_ownership'] = df2['Владение'].apply(num_of_months)
    df2['month_ownership'] = df2['month_ownership'].fillna(df2['month_ownership'].median())


In [ ]:
test.drop(['month_ownership'], axis=1, inplace=True )

In [ ]:
test.sample(4)

In [ ]:
test = pd.concat((test, df2['month_ownership']), axis = 1)
test.sample(4)

In [ ]:
check_df_before_merg(train, test,0)

# Слияние датасетов

In [ ]:
train['Train'] = 1 # помечаем где у нас трейн
test['Train'] = 0 # помечаем где у нас тест

df = train.append(test, sort=False).reset_index(drop=True)
df.sample()

In [ ]:
df.drop(['car_url', 'sell_id' ,'image', 'Состояние', 'Владение'], axis=1, inplace=True )

In [ ]:
df['parsing_unixtime'] = pd.to_datetime(df['parsing_unixtime'], unit='ms')

In [ ]:
df.info()

## Проанализирут столбцы по содержанию

In [ ]:
col_names = df.columns
print(col_names)

In [ ]:
def describe_collumns(df):
    list_of_names = list(df.columns)
    temp_dict = {}
    temp_dict['имя признака'] = list_of_names
    temp_dict['тип'] = df.dtypes
    temp_dict['# пропусков(NaN)'] = df.isnull().sum().values 
    temp_dict['# уникальных'] = df.nunique().values
    
    temp_df = pd.DataFrame.from_dict(temp_dict, orient='index')
    display(temp_df.T)

    return

In [ ]:
describe_collumns(df)

In [ ]:
df[df.modelDate.isnull()==True]

Единичные пропуски содержатся в строке 18700 и 58979. Принято решение удалить эти строки

In [ ]:
df.drop([18700],inplace=True )
df.drop([58979],inplace=True )

In [ ]:
describe_collumns(df)

In [ ]:
def uniq_nan (df,i):
# Уникальные значения, пропущенные значения 
    column_type_numeric = False
    # тип колонки
    if df[col_names[i]].dtype == 'O':
        print("Столбец:", col_names[i], "не является числовым")
    elif df[col_names[i]].dtype == "int64" or df[col_names[i]].dtype == "float64":
        column_type_numeric = True        
        print("Столбец:", col_names[i], "является числовым")
    else:
        print('Признак', col_names[i], 'имеет тип', df[col_names[i]].dtype)
    #print("Столбец:", c)    
    print(pd.DataFrame(df[col_names[i]].unique()))    
    print("Пропущено значений:", df[col_names[i]].isna().sum())
    print(df[col_names[i]].value_counts())

### Пропуски

Пропуски отмечены в 9 колонках: 3,4,7,8,16,22,23,27
в колонке 27 содержаться значения цены

In [ ]:
col_names[3]

In [ ]:
df[col_names[3]]


In [ ]:
#Решено заполнить значением "unknown"
df[col_names[3]] = df[col_names[3]].fillna('unknown')
df[col_names[3]] = df[col_names[3]].str.replace("{'id': '0'}",'unknown')

In [ ]:
df[col_names[3]].isnull().sum()

In [ ]:
col_names[4]

In [ ]:
df[col_names[4]]

In [ ]:
#Решено заполнить значением "unknown"
df[col_names[4]] = df[col_names[4]].fillna('unknown')

In [ ]:
df[col_names[4]].isnull().sum()

In [ ]:
col_names[7]

In [ ]:
#Решено заполнить значением "unknown"
df[col_names[7]] = df[col_names[7]].fillna('unknown')
df[col_names[7]] = df[col_names[7]].str.replace("{}",'unknown')

In [ ]:
df[col_names[7]].isnull().sum()

In [ ]:
col_names[8]

In [ ]:
df[col_names[8]].unique()

In [ ]:
df[df[col_names[8]].isnull()==True]

In [ ]:
df[col_names[8]] = df[col_names[8]].fillna('бензин')

In [ ]:
df[col_names[8]].isnull().sum()

In [ ]:
col_names[16]

In [ ]:
df[col_names[16]] = df[col_names[16]].fillna('RUB')

In [ ]:
col_names[22]

In [ ]:
df[col_names[22]].unique()

In [ ]:
#Решено заполнить значением "unknown"
df[col_names[22]] = df[col_names[22]].fillna('unknown')

In [ ]:
df[col_names[22]].isnull().sum()

In [ ]:
col_names[23]

In [ ]:
df[col_names[23]].unique()

In [ ]:
#Решено заполнить значением "unknown"
df[col_names[23]] = df[col_names[23]].fillna('unknown')

In [ ]:
df[col_names[23]].isnull().sum()

In [ ]:
col_names[27]

In [ ]:
df[col_names[27]] = df[col_names[27]].fillna(0)

In [ ]:
df[col_names[27]].isnull().sum()

In [ ]:
describe_collumns(df)

In [ ]:
df['engineDisplacement'] = df['engineDisplacement'].fillna(df['engineDisplacement'].median())

In [ ]:
df['engineDisplacement'].isnull().sum()

In [ ]:
def descr_col(df,i):
    display(df[i].describe())
    display(df[i].value_counts())

In [ ]:
descr_col(df,'enginePower')

In [ ]:
 df.enginePower = df.enginePower.apply(lambda x: str(x).replace('N12', '')).astype(float)

In [ ]:
descr_col(df,'enginePower')


In [ ]:
descr_col(df,'engineDisplacement')

Добавление новых признаков

In [ ]:
df.productionDate

In [ ]:
df['model_age'] = 2020 - df.modelDate
df['car_age'] = 2020 - df.productionDate
df['age_ratio'] = df['car_age']/df['model_age']
df['age_ratio'].fillna(1.0, inplace = True)

In [ ]:
describe_collumns(df)

In [ ]:
df.parsing_unixtime

## Разделим данные на группы переменных:¶


In [ ]:
df.columns

In [ ]:
time_cols = ['parsing_unixtime'] # временная переменная
cat_cols = ['bodyType', 'brand', 'color', 'fuelType', 'model_name', 
         'vehicleTransmission', 'vendor','Привод','Руль'] # категориальные переменные
num_cols = ['modelDate', 'productionDate', 'mileage', 'numberOfDoors',
            'enginePower','engineDisplacement','model_age', 'car_age', 'age_ratio'] # числовые переменные

## Числовые признаки

In [ ]:
for i in num_cols:
    plt.figure()
    sns.distplot(df[i])
    plt.title(i)
    plt.show()

In [ ]:
# Посмотрим корреляцию числовых признаков
correlation = df[num_cols].corr()
plt.figure(figsize=(10, 6))
sns.heatmap(correlation, annot=True, cmap='coolwarm')

Из графика видно:
+0.97 между годом выпуска и годом модели и новыми признаками. Для анализа оставим car age
-0.75 между пробегом и годом производства. Год производства удалим

In [ ]:
for col in cat_cols:
    print(col, df[col].nunique(), '\n', df[col].value_counts()[:6], '\n')

In [ ]:
df.sample()

## Data Preprocessing

In [ ]:
df.columns

In [ ]:
columns=['bodyType', 'brand', 'color', 'engineDisplacement', 'enginePower', 'fuelType',
       'mileage', 'numberOfDoors', 'vehicleConfiguration', 'vehicleTransmission', 'vendor',
       'Владельцы', 'ПТС', 'Привод', 'Руль', 'month_ownership', 'car_age', 'age_ratio','Train']

In [ ]:
df_cars = df[columns]

In [ ]:
y = df['price'][df['Train'] == 1]

In [ ]:
y.isnull().sum()

## Label Encoding

In [ ]:

data = df_cars

In [ ]:
for colum in ['bodyType', 'brand','color', 'fuelType',
              'vehicleTransmission', 'vendor',
       'Владельцы', 'ПТС', 'Привод', 'Руль']:
    data[colum] = data[colum].astype('category').cat.codes

In [ ]:
data.info()

In [ ]:
X = data.query('Train == 1').drop(['Train'], axis=1)
X_sub = data.query('Train == 0').drop(['Train'], axis=1)

## Train Split

In [ ]:
VAL_SIZE

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

# Model 1: Создадим "наивную" модель 
Эта модель будет предсказывать среднюю цену по модели двигателя (engineDisplacement). 
C ней будем сравнивать другие модели.




In [ ]:
tmp_train = X_train.copy()
tmp_train['price'] = y_train

In [ ]:
# Находим median по экземплярам engineDisplacement в трейне и размечаем тест
predict = X_test['engineDisplacement'].map(tmp_train.groupby('engineDisplacement')['price'].median())

#оцениваем точность
print(f"Точность наивной модели по метрике MAPE: {(mape(y_test, predict.values))*100:0.2f}%")

# # Model 2 : CatBoost
![](https://pbs.twimg.com/media/DP-jUCyXcAArRTo.png:large)   


У нас в данных практически все признаки категориальные. Специально для работы с такими данными была создана очень удобная библиотека CatBoost от Яндекса. [https://catboost.ai](http://)     
На данный момент **CatBoost является одной из лучших библиотек для табличных данных!**

#### Полезные видео о CatBoost (на русском):
* [Доклад про CatBoost](https://youtu.be/9ZrfErvm97M)
* [Свежий Туториал от команды CatBoost (практическая часть)](https://youtu.be/wQt4kgAOgV0) 

In [ ]:
columns=['brand', 'color', 'engineDisplacement', 'enginePower', 'fuelType',
       'mileage', 'numberOfDoors', 'vehicleTransmission', 'vendor',
       'Владельцы', 'ПТС', 'Привод', 'Руль', 'month_ownership', 'car_age', 'age_ratio','Train']

In [ ]:
df_cars = df[columns]
data1 = df_cars

In [ ]:
for colum in ['brand','color', 'fuelType','vendor','Владельцы', 'ПТС', 'Привод', 'Руль',
              'vehicleTransmission', 'month_ownership','car_age']:
    data1[colum] = data1[colum].astype('category').cat.codes

In [ ]:
data1

In [ ]:
X1 = data1.query('Train == 1').drop(['Train'], axis=1)
X1_sub = data1.query('Train == 0').drop(['Train'], axis=1)

In [ ]:
X1

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X1, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

## Fit

In [ ]:
model = CatBoostRegressor(iterations = ITERATIONS,
                          learning_rate = LR,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE']
                         )
model.fit(X_train, y_train,
         #cat_features=cat_features_ids,
         eval_set=(X_test, y_test),
         verbose_eval=100,
         use_best_model=True,
         plot=True
         )

In [ ]:
model.save_model('catboost_single_model_baseline.model')

In [ ]:
predict = model.predict(X_test)

# оцениваем точность
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict))*100:0.2f}%")

Вот так просто со старта, даже не трогая сами данные и не подбирая настройки catboosta, получаем модель с уровнем ошибки в 17%!

# Random Forest

In [ ]:
rf = RandomForestRegressor(random_state=42)

rf.fit(X_train, y_train)
test_predict = rf.predict(X_test)
print(f"Test MAPE: {mape(y_test, test_predict)*100:0.3f}%")

# Bagging

In [ ]:
tree = DecisionTreeRegressor(max_features=int(X_train.shape[1]/3), max_depth=50)
tree.fit(X_train, y_train)
test_predict = tree.predict(X_test)
print(f"DecisionTreeRegressor Test MAPE: {mape(y_test, test_predict)*100:0.3f}%")

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X1, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

# GradientBoosting

In [ ]:
gb = GradientBoostingRegressor(max_depth=7,n_estimators=1000,learning_rate=0.1,random_state=42)
gb.fit(X_train, y_train)
test_predict = gb.predict(X_test)
print(f"GradientBoostingRegressor Test MAPE: {mape(y_test, test_predict)*100:0.3f}%")

# Stacking

In [ ]:
test = data1.query('Train == 0').drop(['Train'], axis=1)
train = data1.query('Train == 1').drop(['Train'], axis=1)

In [ ]:

X = train.values
y = y.values
X_pred = test.values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

In [ ]:
cv = KFold(n_splits=5, shuffle=True, random_state=42)

def compute_metric(regr, X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test):
    regr.fit(X_train, y_train)
    y_test_pred = regr.predict(X_test)
    return np.round(mape(y_test, y_test_pred)*100, 3)

In [ ]:

def compute_meta_feature2(regr, X_train, X_test, y_train, cv, X_pred):    
    
    X_meta_train = np.zeros_like(y_train, dtype=np.float32)    

    splits = cv.split(X_train)
    for train_fold_index, predict_fold_index in splits:
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]
        
        folded_regr = clone(regr)
        folded_regr.fit(X_fold_train, y_fold_train)
        
        X_meta_train[predict_fold_index] = folded_regr.predict(X_fold_predict)
    
    meta_regr = clone(regr)
    meta_regr.fit(X_train, y_train)
    
    X_meta_test = meta_regr.predict(X_test)
    X_meta_pred = meta_regr.predict(X_pred)
    
    return X_meta_train, X_meta_test, X_meta_pred

In [ ]:
def generate_meta_features2(regr_s, X_train, X_test, y_train, cv, X_pred):
   
    features = [compute_meta_feature2(regr, X_train, X_test, y_train, cv, X_pred) for regr in tqdm(regr_s)]    
    stacked_features_train = np.vstack([features_train for features_train, features_test, features_pred in features]).T
    stacked_features_test = np.vstack([features_test for features_train, features_test, features_pred in features]).T
    stacked_features_pred = np.vstack([features_pred for features_train, features_test, features_pred in features]).T
    return stacked_features_train, stacked_features_test, stacked_features_pred

In [ ]:
stacked_features_train, stacked_features_test, stacked_features_pred = generate_meta_features2([
    RandomForestRegressor(n_estimators=300, min_samples_split=2, min_samples_leaf=1, 
                          max_features='sqrt',max_depth=77, bootstrap=True, random_state=42),
    AdaBoostRegressor(random_state=42),
    ExtraTreesRegressor(random_state=42),
    RandomForestRegressor(random_state=42)], X_train, X_test, y_train, cv, X_pred)

regr = RandomForestRegressor(
 n_estimators=300,
 min_samples_split=2,
 min_samples_leaf=1,
 max_features='sqrt',
 max_depth=77,
 bootstrap=True, 
 random_state=42)

print(f'Stacking MAPE = {compute_metric(regr, X_train=stacked_features_train, y_train=y_train, X_test=stacked_features_test, y_test=y_test)}%')

# Submission

In [ ]:
predict_submission = model.predict(X1_sub)
predict_submission

In [ ]:
sample_submission['price'] = predict_submission
sample_submission.to_csv(f'submission_v{VERSION}.csv', index=False)
sample_submission.head(10)